In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

In [2]:
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')

In [3]:
data.head(2)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000


In [4]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    num_train = train.select_dtypes(include=['int','float'])
    num_test = test.select_dtypes(include=['int', 'float'])
    #return the column names and type as series
    
    features = num_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    prediction_train = lr.predict(train[features])
    prediction_test = lr.predict(test[features])
    mse_train = mean_squared_error(prediction_train, train['SalePrice'])
    mse_test = mean_squared_error(prediction_test, test['SalePrice'])
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)
    return rmse_test

transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

# Feature Engineer

Feature Engineering
Handle missing values:
All columns:
Drop any with 5% or more missing values for now.
Text columns:
Drop any with 1 or more missing values for now.
Numerical columns:
For columns with missing values, fill in with the most common value in that column

In [5]:
data.isnull().sum()

Order                0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       490
Lot Area             0
Street               0
Alley             2732
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        23
Mas Vnr Area        23
Exter Qual           0
Exter Cond           0
                  ... 
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual

In [6]:
data.shape

(2930, 82)

1. Drop any with 5% or more missing values for now.

In [7]:
nul_count = data.isnull().sum()

In [8]:
drop_col = nul_count[nul_count > (len(data)/20)].sort_values() 

In [9]:
print(drop_col)

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64


In [10]:
data = data.drop(drop_col.index, axis=1)

2. Text columns: Drop any with 1 or more missing values for now

In [11]:
text_col = data.select_dtypes(include=['object'])
text_nul_count = text_col.isnull().sum().sort_values(ascending=False)
drop_txt_col = text_nul_count[text_nul_count > 0].index
data = data.drop(drop_txt_col, axis=1)

3. Numerical columns: For columns with missing values, fill in with the most common value in that column

In [12]:
num_col = data.select_dtypes(include=['int','float']).columns
data[num_col] = data[num_col].fillna(data.mode())
#fill the missing values in numeric columns with mean of values of that column

In [13]:
data.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
BsmtFin SF 1        1
BsmtFin SF 2        1
                   ..
Central Air         0
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces

Question on how we can create new features. 

In [14]:
years_sold = data['Yr Sold'] - data['Year Built']
drop_year_1 = years_sold[years_sold < 0].index

There is no row that year sold is earlier than built, which is good

In [15]:
year_since_remod = data['Yr Sold'] - data['Year Remod/Add']
drop_year_2 = year_since_remod[year_since_remod < 0].index

In [16]:
print(drop_year_2)

Int64Index([1702, 2180, 2181], dtype='int64')


need to drop these three rows. 

In [17]:
data['years_blt_to_sale'] = years_sold
data['year_since_remod'] = year_since_remod

In [18]:
data = data.drop([1702, 2180, 2181], axis=0)
#drop rows with negative year difference in both columns

data = data.drop(['Year Built', 'Year Remod/Add'], axis=1)
#since we already create new columnes, we can just delete these two original cols

for next step, Drop columns that:

that aren't useful for ML
leak data about the final sale, read more about columns here
for example, PID is just the id for property, the number is not ordinal and is not meaningful for our regression
also, the column associated with final sales includes 
Mo Sold 
Yr Sold             
Sale Type           
Sale Condition      

In [19]:
data = data.drop(['PID','Order'],axis=1)

data = data.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'],axis=1)

In [20]:
#fill up the function we defined earlier
def transform_features(df):
    nul_count = df.isnull().sum()
    drop_col = nul_count[nul_count > (len(df)/20)].sort_values() 
    df = df.drop(drop_col.index, axis=1)
    
    text_col = df.select_dtypes(include=['object'])
    text_nul_count = text_col.isnull().sum().sort_values(ascending=False)
    drop_txt_col = text_nul_count[text_nul_count > 0].index
    df = df.drop(drop_txt_col, axis=1)
    
    num_col = df.select_dtypes(include=['int','float']).columns
    df[num_col] = df[num_col].fillna(df.mode())
    
    years_sold = df['Yr Sold'] - df['Year Built']
    drop_year_1 = years_sold[years_sold < 0].index
    
    year_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    drop_year_2 = year_since_remod[year_since_remod < 0].index
    
    df['years_blt_to_sale'] = years_sold
    df['year_since_remod'] = year_since_remod
    
    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(['Year Built', 'Year Remod/Add'], axis=1)
    
    df = data.drop(['PID','Order'],axis=1)
    df = data.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'],axis=1)
    
    return df

# Feature Selection

Generate a correlation heatmap matrix of the numerical features in the training data set.

In [21]:
#first let's pick numerical columns from all columns
num_features = data.select_dtypes(include=['int','float'])
num_features.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,years_blt_to_sale,year_since_remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


In [22]:
import seaborn as sns
sns.heatmap(num_features)

In [23]:
coeffi = num_features.corr()['SalePrice'].abs().sort_values(ascending=False)
print(coeffi)

SalePrice            1.000000
Overall Qual         0.801206
Gr Liv Area          0.717596
Garage Cars          0.648427
Total Bsmt SF        0.643782
Garage Area          0.641690
1st Flr SF           0.635185
years_blt_to_sale    0.558979
Full Bath            0.546118
year_since_remod     0.534985
Mas Vnr Area         0.513121
TotRms AbvGrd        0.498574
Fireplaces           0.474831
BsmtFin SF 1         0.439051
Wood Deck SF         0.328183
Open Porch SF        0.316262
Half Bath            0.284871
Bsmt Full Bath       0.276486
2nd Flr SF           0.269601
Lot Area             0.267520
Bsmt Unf SF          0.182299
Bedroom AbvGr        0.143916
Enclosed Porch       0.128685
Kitchen AbvGr        0.119760
Screen Porch         0.112280
Overall Cond         0.101540
MS SubClass          0.085128
Pool Area            0.068438
Low Qual Fin SF      0.037629
Bsmt Half Bath       0.035894
3Ssn Porch           0.032268
Misc Val             0.019273
BsmtFin SF 2         0.006001
Name: Sale

In [24]:
#we can just keep features with higher coefficiency with the SalePrice, the threshhold i will pick is features_high = coeffi(coeffi>
#need to filter out non-relavant numerical columns and remove from the dataframe, should not leave highly-corelated columns because that will remove category columns as well
low_col = coeffi[coeffi<0.4]
print(low_col)


Wood Deck SF       0.328183
Open Porch SF      0.316262
Half Bath          0.284871
Bsmt Full Bath     0.276486
2nd Flr SF         0.269601
Lot Area           0.267520
Bsmt Unf SF        0.182299
Bedroom AbvGr      0.143916
Enclosed Porch     0.128685
Kitchen AbvGr      0.119760
Screen Porch       0.112280
Overall Cond       0.101540
MS SubClass        0.085128
Pool Area          0.068438
Low Qual Fin SF    0.037629
Bsmt Half Bath     0.035894
3Ssn Porch         0.032268
Misc Val           0.019273
BsmtFin SF 2       0.006001
Name: SalePrice, dtype: float64


In [25]:
data_remove_low_num = data.drop(low_col.index, axis=1)
print(data_remove_low_num.columns)

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air', '1st Flr SF',
       'Gr Liv Area', 'Full Bath', 'Kitchen Qual', 'TotRms AbvGrd',
       'Functional', 'Fireplaces', 'Garage Cars', 'Garage Area', 'Paved Drive',
       'SalePrice', 'years_blt_to_sale', 'year_since_remod'],
      dtype='object')


In [26]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
#based on documents for columns

Which columns in the data frame should be converted to the categorical data type? All of the columns that can be categorized as nominal variables are candidates for being converted to categorical. Here are some other things you should think about:

If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.

Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).

In [27]:
category_cols = []
for col in nominal_features:
    if col in data.columns:
        category_cols.append(col)

In [28]:
#eliminate the columns with too many varies values, we will define, if a column has more than 10 unique values, we will drop off it
varies_col = data[category_cols].apply(lambda col: len(col.value_counts())).sort_values()
print(varies_col)

Central Air      2
Street           2
Land Contour     4
Lot Config       5
Bldg Type        5
Heating          6
Roof Style       6
Foundation       6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
MS SubClass     16
Exterior 2nd    17
Neighborhood    28
dtype: int64


In [29]:
varies_col_keep = varies_col[(varies_col <= 10) & (varies_col > 4)]
#we don't want columns have too many varies values but also not too focus on 1 or 2 values
varies_col_rem = varies_col.drop(varies_col_keep.index)
print(varies_col_rem)
data = data.drop(varies_col_rem.index, axis=1)

Central Air      2
Street           2
Land Contour     4
Exterior 1st    16
MS SubClass     16
Exterior 2nd    17
Neighborhood    28
dtype: int64


In [30]:
#convert object-type columns into category columns
text_col = data.select_dtypes(include=['object']).columns
for col in text_col:
    data[col] = data[col].astype('category')
    
#create dummies column for category columns
dummy_df = pd.get_dummies(data.select_dtypes(include=['category']))
new_data = pd.concat([data, dummy_df],axis=1)
new_data = new_data.drop(text_col, axis=1)

In [31]:
#fill up the function we defined earlier

def transform_features(df):
    
    nul_count = df.isnull().sum()
    drop_col = nul_count[nul_count > (len(df)/20)].sort_values() 
    df = df.drop(drop_col.index, axis=1)
    
    text_col = df.select_dtypes(include=['object'])
    text_nul_count = text_col.isnull().sum().sort_values(ascending=False)
    drop_txt_col = text_nul_count[text_nul_count > 0]
    df = df.drop(drop_txt_col.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
      
    years_sold = df['Yr Sold'] - df['Year Built']
    drop_year_1 = years_sold[years_sold < 0].index
    
    year_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    drop_year_2 = year_since_remod[year_since_remod < 0].index
    
    df['years_blt_to_sale'] = years_sold
    df['year_since_remod'] = year_since_remod
    
    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(['Year Built', 'Year Remod/Add'], axis=1)
    
    df = data.drop(['PID','Order'],axis=1)
    df = data.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'],axis=1)
    
    return df

def select_features(df):
    
    num_features = df.select_dtypes(include=['int','float'])
    coeffi = num_features.corr()['SalePrice'].abs().sort_values(ascending=False)
    low_col = coeffi[coeffi<0.4]
    data_remove_low_num = df.drop(low_col.index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    category_cols = []
    for col in nominal_features:
        if col in df.columns:
            category_cols.append(col)
    varies_col = df[category_cols].apply(lambda col: len(col.value_counts())).sort_values()
    varies_col_keep = varies_col[(varies_col <= 10) & (varies_col > 4)]
    varies_col_rem = varies_col.drop(varies_col_keep.index)
    df = df.drop(varies_col_rem.index, axis=1)
    text_col = df.select_dtypes(include=['object']).columns
    for col in text_col:
        df[col] = df[col].astype('category')
    
    dummy_df = pd.get_dummies(df.select_dtypes(include=['category']))
    new_data = pd.concat([df, dummy_df],axis=1)
    new_data = new_data.drop(text_col, axis=1)
    
    return df


def train_and_test(df, k=0):
    num_df = df.select_dtypes(include=['int','float'])
    features = num_df.columns.drop('SalePrice')
    #return the column names and type as series
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]
        lr.fit(train[features], train['SalePrice'])
        prediction_train = lr.predict(train[features])
        prediction_test = lr.predict(test[features])
        mse_train = mean_squared_error(prediction_train, train['SalePrice'])
        mse_test = mean_squared_error(prediction_test, test['SalePrice'])
        rmse_train = np.sqrt(mse_train)
        rmse_test = np.sqrt(mse_test)
        return rmse_test

    if k == 1:
        shuffle_df = df.sample(frac=1)
        # Randomize *all* rows (frac=1) from `df`
        fold_one = df[:1460]
        fold_two = df[1460:]
        
        lr.fit(fold_one[features], fold_one['SalePrice'])
        prediction_f2 = lr.predict(fold_two[features])
        mse_f2 = mean_squared_error(prediction_f2, fold_two['SalePrice'])
        rmse_f2 = np.sqrt(mse_f2)
        
        lr.fit(fold_two[features], fold_two['SalePrice'])
        prediction_f1 = lr.predict(fold_one[features])
        mse_f1 = mean_squared_error(prediction_f1, fold_one['SalePrice'])
        rmse_f1 = np.sqrt(mse_f1)
        
        rmse_ave = np.mean([rmse_f1, rmse_f2])
        print(rmse_f1)
        print(rmse_f2)            
        return rmse_ave
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_fold = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train['SalePrice'])
            prediction_test = lr.predict(test[features])
            mse_test = mean_squared_error(prediction_test, test['SalePrice'])
            rmse_test = np.sqrt(mse_test)
            rmse_fold.append(rmse_test)
        print(rmse_fold)
        rmse_ave =  np.mean(rmse_fold)
        return rmse_ave
    


In [32]:
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
transform_data.isnull().sum()

Order                0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       490
Lot Area             0
Street               0
Alley             2732
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        23
Mas Vnr Area        23
Exter Qual           0
Exter Cond           0
                  ... 
Bsmt Full Bath       2
Bsmt Half Bath       2
Full Bath            0
Half Bath            0
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu      1422
Garage Type        157
Garage Yr B